# Simple demonstration of harmonic function operations

In [38]:
import copy
from sympy import sin, symbols, plot, pi
from ipywidgets import interactive, FloatSlider, BoundedFloatText, BoundedIntText, \
                    Checkbox, HBox, VBox, Layout, Label
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = 10, 4
plt.rcParams['legend.loc'] = 'lower right'

## Amplitude, frequency and phase

Manipulates $A \sin(2\pi f t + \phi)$.

In [39]:
def harmonic(A, f, phi):
    t = symbols('t')
    omega=2*pi*f
    phi=phi*pi/180.0
    plot(A*sin(omega*t+phi), adaptive=True, autoscale=False, xlim=(-3*tmax, 3*tmax), ylim=(-1.1*amax, 1.1*amax))

label_layout=Layout(width='100px')
fmax = 3.0
tmax = 2*pi/fmax
amax = 2

Aw = FloatSlider(value=1, min=0, max=amax, step=0.1, readout_format='.1f', continuous_update=False)
AwL = Label('Amplitude ', layout=label_layout)

Fw = FloatSlider(value=1, min=0, max=fmax, step=0.1, readout_format='.2f', continuous_update=True)
FwL = Label('Frequência (Hz) ', layout=label_layout)

Pw = FloatSlider(value=0, min=0, max=360, step=2, readout_format='.1f', continuous_update=False)
PwL = Label('Fase (Graus) ', layout=label_layout)

w = interactive(harmonic, A=Aw, f=Fw, phi=Pw)
lbox = VBox([HBox([AwL, Aw]), HBox([FwL, Fw]),HBox([PwL, Pw])])
h = VBox([lbox, w.children[-1]])
display(h)
w.update()

## Sum of harmonic functions

Plots $A_1\sin(2 \pi f_1 t + \phi_1) + A_2\sin(2\pi f_2 t + \phi_2)$, with $f$ in Hz and $\phi$ in degrees.

In [40]:
t = symbols('t')

def harmonic_sum(A1, f1, phi1, A2, f2, phi2, ntau):
    label1 = "$x_1 = {:4.1f}\, \sin(2 \pi \, {:4.1f} + {:4.1f}$)".format(A1, f1, phi1)
    label2 = "$x_2 = {:4.1f}\, \sin(2 \pi \, {:4.1f} + {:4.1f}$)".format(A2, f2, phi2)
    w1 = 2*pi*f1
    w2 = 2*pi*f2
    phi1 *= pi/180.0
    phi2 *= pi/180.0
    f1 = A1*sin(w1*t+phi1)
    f2 = A2*sin(w2*t+phi2)
    w = min(w1, w2)
    tau = 2*pi/w
    f = f1 + f2
    tend=ntau*tau
    plot(f1, f2, f1+f2, (t, 0, tend), legend=True, adaptive=False, nb_of_points=ntau*30)

A1w = BoundedFloatText(value=1.0, min=0, max=10, step=0.25, description='A1:')
A2w = BoundedFloatText(value=1.0, min=0, max=10, step=0.25, description='A2:')
f1w = BoundedFloatText(value=2.0, min=0, max=5, step=0.25, description='ω1:')
f2w = BoundedFloatText(value=2.0, min=0, max=5, step=0.25, description='ω2:')
phi1w = BoundedFloatText(value=0.0, min=0, max=360, step=5, description='ϕ1 (degree):')
phi2w = BoundedFloatText(value=90, min=0, max=360, step=5, description='ϕ2 (degree):')
ntw = BoundedIntText(value=3, min=1, max=500, step=1, description=u'nº Periods')

w = interactive(harmonic_sum, A1=A1w, f1=f1w, phi1=phi1w, A2=A2w, f2=f2w, phi2=phi2w, ntau=ntw)
controls = HBox( [ VBox(w.children[:3]), VBox(w.children[3:6]), ntw])
d = VBox([controls, w.children[-1]])
display(d)
w.update()

## Special case, triphase sum

Plots $\sin(2\pi t) + \sin(2\pi t + 2\pi/3) + \sin(2\pi t + 4\pi/3)$, incrementally.

In [41]:
s1 = sin(2*pi*t)
s2 = sin(2*pi*t+2*pi/3)
s3 = sin(2*pi*t+4*pi/3)
p1 = plot(s1, (t, 0, 5), legend=True, label='0', show=False) 
p2 = plot(s2, (t, 0, 5), legend=True, label='120', show=False) 
p3 = plot(s3, (t, 0, 5), legend=True, label='240', show=False) 

def triphase_sum(f1, f2, f3, fs):
    ps = 0
    pp = None
    ic(ps, pp)
    for f, p, s in zip([f1, f2, f3], [p1, p2, p3], [s1, s2, s3]):
        if f:
            if not pp: pp = copy.deepcopy(p)
            else: pp.append(p[0])
            ps += s

    if fs:
        psum = plot(ps, (t, 0, 5), legend=True, label='sum', show=False) 
        if not pp: pp = copy.deepcopy(psum)
        else: pp.append(psum[0])

    if pp: pp.show()

f1w = Checkbox(value=True, description='ϕ = 0')
f2w = Checkbox(value=False, description='ϕ = 120')
f3w = Checkbox(value=False, description='ϕ = 240')
fsw = Checkbox(value=True, description='Sum')
w = interactive(triphase_sum, f1=f1w, f2=f2w, f3=f3w, fs=fsw)
controls = HBox(w.children[:-1])
d = VBox([controls, w.children[-1]])
display(d)
w.update()